In [ ]:
#!pip install transformers

In [1]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ECMA 31330 Final Project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

Mounted at /content/drive
['Articles', 'SentimentScores', 'Aakash Gupta Final Project Presentation.gslides', 'Sp500Stocks.csv', 'SPY2015.csv', 'SPY2020.csv', 'SPY-2015_2020.csv', 'SimpleGARCHModel.ipynb', 'DataCleaningPipeline.ipynb', 'NeuralNet.ipynb', 'GARCHFittingPipeline.ipynb', 'LassoModel.ipynb', 'SentimentAnalysisPipeline.ipynb', 'Flowcharts.gslides']


In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import datetime
import yfinance as yf
from datetime import datetime

In [15]:
articles = pd.read_csv(GOOGLE_DRIVE_PATH + "/Articles/filtered2020articles.csv")
articles[100:150]

,Unnamed: 0.1,Unnamed: 0,article_id,article_url,comment_count,year,date,title,datetimeobject
100,786,41221,507,https://www.nytimes.com/video/us/politics/1000...,0,2020,01/01/1975,staten island coronavirus bar protest,1975-01-01
101,788,41207,493,https://www.nytimes.com/video/us/politics/1000...,0,2020,01/01/1975,de blasio coronavirus new york,1975-01-01
102,797,30283,3173,https://www.nytimes.com/video/world/europe/100...,0,2020,01/01/1975,ireland coronavirus lockdown,1975-01-01
103,798,30176,3066,https://www.nytimes.com/video/us/politics/1000...,0,2020,01/01/1975,california newsom coronavirus vaccine,1975-01-01
104,799,30158,3048,https://www.nytimes.com/video/us/politics/1000...,0,2020,01/01/1975,new jersey coronavirus increase,1975-01-01
105,811,29626,2516,https://www.nytimes.com/video/world/europe/100...,0,2020,01/01/1975,london mayor coronavirus restrictions,1975-01-01
106,815,29448,2338,https://www.nytimes.com/video/us/politics/1000...,0,2020,01/01/1975,bill de blasio new york city coronavirus,1975-01-01
107,847,30722,3612,https://www.nytimes.com/video/us/politics/1000...,0,2020,01/01/1975,pelosi coronavirus stimulus negotiations,1975-01-01
108,853,30485,3375,https://www.nytimes.com/video/us/politics/1000...,0,2020,01/01/1975,new york ease coronavirus restrictions,1975-01-01
109,862,29038,1928,https://www.nytimes.com/video/world/europe/100...,0,2020,01/01/1975,boris johnson tiered cotronavirus lockdown,1975-01-01


In [4]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cpu


In [17]:
input_negative = tokenizer("Apple Lays off 10% of workforce", return_tensors='pt')
input_positive = tokenizer("Apple reaches record profits this quarter", return_tensors='pt')
output_negative = model(**input_negative)
output_positive = model(**input_positive)
predictions_negative = torch.nn.functional.softmax(output_negative.logits, dim=-1)
predictions_positive = torch.nn.functional.softmax(output_positive.logits, dim=-1)
print(predictions_negative)
print(predictions_positive)

tensor([[0.0081, 0.9498, 0.0422]], grad_fn=<SoftmaxBackward0>)
tensor([[0.8821, 0.0408, 0.0771]], grad_fn=<SoftmaxBackward0>)


In [5]:
headlines_list = list(articles["title"][3000:3500])
article_id_list = list(articles['article_id'][3000:3500])
dates_list = list(articles["datetimeobject"][3000:3500])
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')

In [6]:
inputs

{'input_ids': tensor([[  101,  2647,  2430,  ...,     0,     0,     0],
        [  101,  8398, 21887,  ...,     0,     0,     0],
        [  101, 21887, 23350,  ...,     0,     0,     0],
        ...,
        [  101, 21442, 11705,  ...,     0,     0,     0],
        [  101,  2859, 26269,  ...,     0,     0,     0],
        [  101,  2859,  5635,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()
table = {'Headline':headlines_list,
         "article_id": article_id_list,
         "date": dates_list,
         "Positive":positive,
         "Negative":negative,
         "Neutral":neutral}
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])
df['date'] = dates_list
df['article_id'] = article_id_list
df.to_csv(GOOGLE_DRIVE_PATH + "/SentimentScores/SentimentProsusAI_2020_3500.csv")

#finished the first 13000 articles

In [ ]:
df

,Headline,Positive,Negative,Neutral,date,article_id
0,european central bank rates coronavirus,0.126560,0.072396,0.801045,2020-12-03,10567
1,trump coronavirus europe allies,0.034345,0.027543,0.938113,2020-12-03,10590
2,coronavirus airlines trump europe,0.050578,0.030911,0.918510,2020-12-03,10514
3,trudeau wife coronavirus,0.035673,0.115505,0.848822,2020-12-03,10646
4,strategies coronavirus pandemic different,0.042419,0.541763,0.415818,2020-12-03,10571
...,...,...,...,...,...,...
433,merkel covid new year speech,0.098676,0.026140,0.875184,2020-12-31,4116
434,largely free of the virus new zealand could pa...,0.254070,0.024834,0.721097,2020-12-31,4142
435,merkel covid new year speech,0.098676,0.026140,0.875184,2020-12-31,4116
436,china tariffs exclusions expire,0.022958,0.159789,0.817253,2020-12-31,4083
